In [ ]:
import mediapipe as mp
import numpy as np
import cv2
import math
import random
import joblib
import sys
import csv
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Stone - 0, Paper - 1, Scissors - 2
mpDraw = mp.solutions.drawing_utils
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands = 1, min_detection_confidence = 0.65)

In [ ]:
def stone_paper_scissors(listX, listY, listZ):
    if(len(listX) == 21 * factor and len(listY) == 21 * factor):
            tiltm = 0
            for k in range(factor):
                midX = listX[k * 21 + 9] + listX[k * 21 + 13] // 2
                midY = listY[k * 21 + 9] + listY[k * 21 + 13] // 2
                if(listX[k * 21] - midX != 0):
                    val = math.atan((listY[k * 21] - midY) / (listX[k * 21] - midX))
                handStartX, handStartY = listX[k * 21 + 5], listY[k * 21 + 5]
                handEndX, handEndY = listX[k * 21 + 17], listY[k * 21 + 17]
                if((val < 0.22 and val > -0.64) or (val < 1.5 and val > 1.45)):
                    if(run):
                        for i in range(1 + 21 * k, 21 * (k + 1), 4):
                            Ix, Iy = listX[i], listY[i]
                            IIx, IIy = listX[i + 1], listY[i + 1]
                            IIIx, IIIy = listX[i + 2], listY[i + 2]
                            IVx, IVy = listX[i + 3], listY[i + 3]
                            if(IIy - IIIy > 0):
                                if(i == 1 + 21 * k):
                                    if(handStartX > handEndX):
                                        if(((IVx > handStartX or IVx < handEndX) and IIx < IIIx)):
                                            tiltm = tiltm + 1
                                    else:
                                        if(((IVx < handStartX or IVx > handEndX) and IIx > IIIx)):
                                            tiltm = tiltm + 1
                                else:
                                    tiltm = tiltm + 1
                elif(listX[21 * k] - listX[21 * k + 9] < 0):
                    if(run):
                        for i in range(1 + 21 * k, 21 * (k + 1), 4):
                            Ix, Iy = listX[i], listY[i]
                            IIx, IIy = listX[i + 1], listY[i + 1]
                            IIIx, IIIy = listX[i + 2], listY[i + 2]
                            IVx, IVy = listX[i + 3], listY[i + 3]
                            if(IIx - IIIx < 0 & listX[k] - listX[k + 9] < 0):
                                if(i == 1 + 21 * k):
                                    if(handStartY > handEndY):
                                        if(((IVy > handStartY or IVy < handEndY) and IIy < IIIy)):
                                            tiltm = tiltm + 1
                                    else:
                                        if(((IVy < handStartY or IVy > handEndY) and IIy > IIIy)):
                                            tiltm = tiltm + 1
                                else:
                                    if(IIx - IIIx > 0):
                                        tiltm = tiltm + 1
                            else:
                                if(i == 1 + 21 * k):
                                    if(handStartY < handEndY):
                                        if(((IVy < handStartY or IVy > handEndY) and IIy > IIIy)):
                                            tiltm = tiltm + 1
                                    else:
                                        if(((IVy > handStartY or IVy < handEndY) and IIy < IIIy)):
                                            tiltm = tiltm + 1
                                else:
                                    if(IIx - IIIx < 0):
                                        tiltm = tiltm + 1
                else:
                    if(run):
                        for i in range(1 + 21 * k, 21 * (k + 1), 4):
                            Ix, Iy = listX[i], listY[i]
                            IIx, IIy = listX[i + 1], listY[i + 1]
                            IIIx, IIIy = listX[i + 2], listY[i + 2]
                            IVx, IVy = listX[i + 3], listY[i + 3]
                            if(IIx - IIIx > 0 & listX[k] - listX[k + 9] > 0):
                                if(i == 1 + 21 * k):
                                    if(handStartY < handEndY):
                                        if(((IVy < handStartY or IVy > handEndY) and IIy > IIIy)):
                                            tiltm = tiltm + 1
                                    else:
                                        if(((IVy > handStartY or IVy < handEndY) and IIy < IIIy)):
                                            tiltm = tiltm + 1
                                else:
                                    if(IIx - IIIx > 0):
                                        tiltm = tiltm + 1
                            else:
                                if(i == 1 + 21 * k):
                                    if(handStartY > handEndY):
                                        if(((IVy > handStartY or IVy < handEndY) and IIy < IIIy)):
                                            tiltm = tiltm + 1
                                    else:
                                        if(((IVy < handStartY or IVy > handEndY) and IIy > IIIy)):
                                            tiltm = tiltm + 1
                                else:
                                    if(IIx - IIIx > 0):
                                        tiltm = tiltm + 1
                return tiltm

In [ ]:
def response(prevMoveP, prevMoveC):
    if prevMoveP == prevMoveC:
        return 'Tie'
    elif prevMoveP == 0:
        if prevMoveC == 1:
            return 'Lost'
        else:
            return 'Win'
    elif prevMoveP == 1:
        if prevMoveC == 0:
            return 'Win'
        else:
            return 'Lost'
    else:
        if prevMoveC == 1:
            return 'Win'
        else:
            return 'Lost'

In [ ]:
def play(text, listMoves, listMovesComputer, scoreP, scoreC):
    if(len(listMoves) != 0):
        prevMovePlayer = listMoves[len(listMoves) - 1]
    if(len(listMovesComputer) != 0):
        prevMoveComputer = listMovesComputer[len(listMovesComputer) - 1]
    res = response(prevMovePlayer, prevMoveComputer)
    y_pred = [-1]
    try:
        dataset = pd.read_csv('stonepaperscissors.csv')
        X = dataset.iloc[:, 1:5].values
        Y = dataset.iloc[:, 5].values 
        if res ==  'Win':
            scoreP = scoreP + 1
            predict_x = [[prevMovePlayer, prevMoveComputer, scoreP, scoreC]]
            x = [[len(X) + 1, prevMovePlayer, prevMoveComputer, scoreP, scoreC, 0]]
        elif res == 'Lost':
            scoreC = scoreC + 1
            predict_x = [[prevMovePlayer, prevMoveComputer, scoreP, scoreC]]
            x = [[len(X) + 1, prevMovePlayer, prevMoveComputer, scoreP, scoreC, 0]]
        else:
            predict_x = [[prevMovePlayer, prevMoveComputer, scoreP, scoreC]]
            x = [[len(X) + 1, prevMovePlayer, prevMoveComputer, scoreP, scoreC, 0]]
        print(len(X))
        if(len(X) % 50 == 0):
            from sklearn.neighbors import KNeighborsClassifier
            classifier = KNeighborsClassifier(n_neighbors = max(1, int(0.12 * len(X))), metric = 'minkowski', p = 1)
            classifier.fit(X, Y)
            y_pred = classifier.predict(predict_x)
            with open('Model.pkl', 'wb') as file:
                joblib.dump(classifier, file)
        elif(len(X) > 50):
            with open('Model.pkl', 'rb') as file:
                joblib_model = joblib.load(file)
            y_pred = joblib_model.predict(predict_x)
        Y[len(Y) - 1] = prevMovePlayer
        dataset['Next Move Player'] = Y
        df = pd.DataFrame(x, columns =['Game Number', 'Previous Move Player', 'Previous Move Computer', 'Player Score', 'Computer Score', 'Next Move Player'])
        dataset = dataset.append(df, ignore_index = False)
        dataset.to_csv('stonepaperscissors.csv', index = False)
    except :
        if res ==  'Win':
            x = [[1, prevMovePlayer, prevMoveComputer, 1, 0, 0]]
        elif res == 'Lost':
            x = [[1, prevMovePlayer, prevMoveComputer, 0, 1, 0]]
        else:
            x = [[1, prevMovePlayer, prevMoveComputer, 0, 0, 0]]
        df = pd.DataFrame(x, columns =['Game Number', 'Previous Move Player', 'Previous Move Computer', 'Player Score', 'Computer Score', 'Next Move Player'])
        df.to_csv('stonepaperscissors.csv', index = False)
    finally:
        return res, listMoves, listMovesComputer, scoreP, scoreC, y_pred[0]

In [193]:
cam = cv2.VideoCapture(0)
listX = []
listY = []
listZ = []
listMoves = []
listMovesComputer = []
possibleMoves = [0, 1, 2] #'Stone', 'Paper', 'Scissors'
screenWidth = 480
screenHeight = 640
scoreP = 0
scoreC = 0
exitLoop = False
stone = cv2.imread('rocks.png')
scissors = cv2.imread('scissors.png')
paper = cv2.imread('paper.png')
imageC = paper.copy()
imageC = cv2.resize(imageC, (320, 400))
game = False
while True:
    ret, frame = cam.read()
    frame = cv2.flip(frame, 1)
    frame1 = frame.copy()
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(img)
    k = 0
    run = False
    if results.multi_hand_landmarks:
        for handLandMarks in results.multi_hand_landmarks:
            #mpDraw.draw_landmarks(frame, handLandMarks, mpHands.HAND_CONNECTIONS)
            factor = len(results.multi_hand_landmarks)
            for lm in handLandMarks.landmark:
                h, w, c = frame.shape
                listX.append(int(lm.x * w))
                listY.append(int(lm.y * h))
                listZ.append(lm.z)
                run = True
            listX = listX[len(listX) - 21 * factor:]
            listY = listY[len(listY) - 21 * factor:]
            listZ = listZ[len(listZ) - 21 * factor:]
            tiltm = stone_paper_scissors(listX, listY, listZ)
            if(tiltm == 2):
                text = 2#'Scissors'
                image = scissors.copy()
                image = cv2.resize(image, (320, 400))
                #frame = cv2.putText(frame, 'Scissors', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 2, cv2.LINE_AA)
            elif(tiltm == 5 or tiltm == 4):
                text = 1#'Paper'
                image = paper.copy()
                image = cv2.resize(image, (320, 400))
                #frame = cv2.putText(frame, 'Paper', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 2, cv2.LINE_AA)
            elif(tiltm == 0 or tiltm == 1):
                text = 0#'Stone'
                image = stone.copy()
                image = cv2.resize(image, (320, 400))
                #frame = cv2.putText(frame, 'Stone', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 2, cv2.LINE_AA)
            if cv2.waitKey(1) & 0xFF == ord('p'):
                if(run):
                    listMoves.append(text)
                    if(len(listMoves) > len(listMovesComputer)):
                        listMovesComputer.append(possibleMoves[abs(random.randint(0, 10000)) % 3])
                    result, listMoves, listMovesComputer, scoreP, scoreC, y = play(text, listMoves, listMovesComputer, scoreP, scoreC)
                    print(result, " ", listMoves[len(listMoves) - 1], " ", listMovesComputer[len(listMovesComputer) - 1])
                    if y != -1:
                        if y == 0:
                            listMovesComputer.append(1)
                        elif y == 1:
                            listMovesComputer.append(2)
                        elif y == 2:
                            listMovesComputer.append(0)
                    if(listMovesComputer[len(listMovesComputer) - 1] == 0):
                        imageC = stone.copy()
                        imageC = cv2.resize(imageC, (320, 400))
                        game = True
                    elif(listMovesComputer[len(listMovesComputer) - 1] == 1):
                        imageC = paper.copy()
                        imageC = cv2.resize(imageC, (320, 400))
                        game = True
                    elif(listMovesComputer[len(listMovesComputer) - 1] == 2):
                        imageC = scissors.copy()
                        imageC = cv2.resize(imageC, (320, 400))
                        game = True
                    if scoreP == 5 or scoreC == 5:
                        exitLoop = True
                else:
                    print('Hand not detected')
    if(run):
        imageC = cv2.putText(imageC, 'Comp: ' + str(scoreC), (150, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1, cv2.LINE_AA)
        image = cv2.putText(image, 'Player: ' + str(scoreP), (150, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1, cv2.LINE_AA)
        final = np.hstack((image, imageC))
        if(game):
            final = cv2.putText(final, 'Game', (200, 240), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1, cv2.LINE_AA)
            game = False
        cv2.imshow('Frame', final)
    else:
        cv2.putText(frame1, 'Hand not detected', (200, 240), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1, cv2.LINE_AA)
        cv2.imshow('Frame', frame1)
    run = False
    if (cv2.waitKey(1) & 0xFF == ord('q')) or exitLoop:
        break
cam.release()
cv2.destroyAllWindows()

117
Lost   2   0
118
Lost   0   1
119
Lost   2   0
120
Tie   2   2
121
Tie   1   1
122
Win   1   0
123
Win   1   0
124
Lost   2   0
125
Win   2   1
126
Lost   1   2
